<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/quantconnect_lightgbm_ups_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# QuantConnect version of LightGBM model training and trading
from AlgorithmImports import *
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class LightGBMQuantConnect(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2024, 1, 1)
        self.SetCash(100000)

        self.symbol = self.AddEquity("UPS", Resolution.Hour).Symbol
        self.lookback = 500
        self.train_model_ready = False
        self.last_action = 0
        self.features = ["SMA_50", "EMA_20", "RSI", "MACD", "Signal_Line", "ATR", "OBV", "CCI"]

        self.Schedule.On(self.DateRules.EveryDay(self.symbol),
                         self.TimeRules.Every(timedelta(hours=1)),
                         self.TradeLogic)

    def OnData(self, data):
        if not self.train_model_ready:
            history = self.History(self.symbol, self.lookback, Resolution.Hour).dropna()
            if history.empty:
                self.Debug("No history yet.")
                return
            df = history.reset_index().rename(columns={"time": "Datetime", "close": "Close", "high": "High", "low": "Low", "volume": "Volume"})
            df = self.AddFeatures(df)
            df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
            df.dropna(inplace=True)
            X = df[self.features]
            y = df['Target']

            self.scaler = MinMaxScaler()
            X_scaled = self.scaler.fit_transform(X)

            self.model = LGBMClassifier(n_estimators=100)
            self.model.fit(X_scaled, y)
            self.train_model_ready = True
            self.Debug("Model trained.")

    def AddFeatures(self, df):
        df['SMA_50'] = df['Close'].rolling(window=50).mean()
        df['EMA_20'] = df['Close'].ewm(span=20).mean()
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        rs = gain / (loss + 1e-6)
        df['RSI'] = 100 - (100 / (1 + rs))
        df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
        df['Signal_Line'] = df['MACD'].ewm(span=9).mean()
        df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
        df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
        tp = (df['High'] + df['Low'] + df['Close']) / 3
        df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
        df.dropna(inplace=True)
        return df

    def TradeLogic(self):
        if not self.train_model_ready:
            return

        history = self.History(self.symbol, 50, Resolution.Hour).dropna()
        if history.empty:
            return

        df = history.reset_index().rename(columns={"time": "Datetime", "close": "Close", "high": "High", "low": "Low", "volume": "Volume"})
        df = self.AddFeatures(df)
        latest = df.iloc[-1:]

        X_live = self.scaler.transform(latest[self.features])
        pred = self.model.predict(X_live)[0]

        if pred == 1 and self.last_action != 1:
            self.SetHoldings(self.symbol, 1.0)
            self.last_action = 1
            self.Debug("BUY")
        elif pred == 0 and self.last_action != -1:
            self.Liquidate(self.symbol)
            self.last_action = -1
            self.Debug("SELL")
